In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
#TF-IDF
from pyspark.ml.feature import HashingTF,IDF, Tokenizer

In [0]:
sentenceData = spark.createDataFrame([
    (0.0,'Hi I heard about Spark'),
    (0.0,'I wish java could use case classes'),
    (0.1,'Logistic regression models are neat')
],['label','sentence'])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  0.1|Logistic regressi...|
+-----+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')
words_data = tokenizer.transform(sentenceData)
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish java could...|[i, wish, java, c...|
|  0.1|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [0]:
#Getting TF
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)
featurized_data.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|
|  0.0|I wish java could...|[i, wish, java, c...|(262144,[19036,20...|
|  0.1|Logistic regressi...|[logistic, regres...|(262144,[46243,58...|
+-----+--------------------+--------------------+--------------------+



In [0]:
#IDF
idf = IDF(inputCol='rawFeatures',outputCol='features')
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select('label','features').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|0.1 

In [0]:
#Count Vectorizer
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0,'a b c'.split(' ')),
    (1,'a b b c a'.split(' '))
],['id','words'])

df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [0]:
cv = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3,minDF=2)
model = cv.fit(df)
results = model.transform(df)
results.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

